In [1]:
from matplotlib import pyplot as plt
import math
import pandas as pd
import numpy as np
import seaborn as sns
import category_encoders as ce
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import (
    make_scorer,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVR
from sklearn.utils import shuffle
from tqdm import tqdm as progress_bar

In [2]:
pd.set_option("display.max_columns", 105)

## Load the data

In [3]:
data = pd.read_csv("C:/Users/meykevandenbos/Documents/Meyke persoonlijk/Techionista/ML/House Prices/cleanPlus_data.csv")

In [4]:
del data['Unnamed: 0']

In [5]:
data.head()

,Id,MSSubClass,LotArea,LotShape,LandContour,LandSlope,OverallQual,OverallCond,MasVnrArea,ExterQual,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,SalePrice,TotalSF,TotalPorchArea,LotArea (box-cox),1stFlrSF (box-cox),GrLivArea (box-cox),TotalSF (box-cox),SalePrice (box-cox),has_basement,has_fireplace,has_garage,has_2ndfloor,AgeHouse,AgeRemod,hood_Blmngtn,hood_Blueste,hood_BrDale,hood_BrkSide,hood_ClearCr,hood_CollgCr,hood_Crawfor,hood_Edwards,hood_Gilbert,hood_IDOTRR,hood_MeadowV,hood_Mitchel,hood_NAmes,hood_NPkVill,hood_NWAmes,hood_NoRidge,hood_NridgHt,hood_OldTown,hood_SWISU,hood_Sawyer,hood_SawyerW,hood_Somerst,hood_StoneBr,hood_Timber,hood_Veenker,park,street,townHouse,multi_family_home,PConc,CBlock,BrkTil,SaleTypeNew,SaleCondPartial,SaleCondAbnormal,outlier
0,1,60,8450,3,3,0,7,5,196.0,2,2,4,3,2,706.0,6,0.0,150.0,856.0,0,1,1,856,854,0,1710,1.0,0.0,2,1,3,1,2,8,6,0,3,0,2,2.0,548.0,5,5,1,0,61,0,0,0,0,3,4,0,208500.0,2566.0,61,17.233406,6.752270,7.444249,24.274473,12.247694,1,0,1,1,5,5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,20,9600,3,3,0,6,8,0.0,3,2,4,1,0,978.0,6,0.0,284.0,1262.0,0,1,1,1262,0,0,1262,0.0,1.0,2,0,3,1,3,6,6,1,5,0,2,2.0,460.0,5,5,1,298,0,0,0,0,0,3,4,0,181500.0,2524.0,298,17.650449,7.140453,7.140453,24.158678,12.109011,1,1,1,0,31,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0
2,3,60,11250,0,3,0,7,5,162.0,2,2,4,2,2,486.0,6,0.0,434.0,920.0,0,1,1,920,866,0,1786,1.0,0.0,2,1,3,1,2,6,6,1,5,0,2,2.0,608.0,5,5,1,0,42,0,0,0,0,3,4,0,223500.0,2706.0,42,18.178584,6.824374,7.487734,24.650449,12.317167,1,1,1,1,7,6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,70,9550,0,3,0,7,5,0.0,3,4,1,3,0,216.0,6,0.0,540.0,756.0,2,1,1,961,756,0,1717,1.0,0.0,1,0,3,1,2,7,6,1,2,1,3,3.0,642.0,5,5,1,0,35,272,0,0,0,3,4,0,140000.0,2473.0,307,17.633246,6.867974,7.448334,24.016107,11.849398,1,1,1,1,91,36,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,5,60,14260,0,3,0,8,5,350.0,2,2,4,0,2,655.0,6,0.0,490.0,1145.0,0,1,1,1145,1053,0,2198,1.0,0.0,2,1,4,1,2,9,6,1,5,0,2,3.0,836.0,5,5,1,192,84,0,0,0,0,3,4,0,250000.0,3343.0,276,18.988661,7.043160,7.695303,26.196741,12.429216,1,1,1,1,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [90]:
data_train = data.loc[(data['SalePrice'].notna() & (data['outlier'] != 1))]
data_toPredict = data.loc[(data['SalePrice'].isna())]

In [91]:
data_train.info()
data_toPredict.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1449 entries, 0 to 1459
Columns: 103 entries, Id to outlier
dtypes: float64(16), int64(87)
memory usage: 1.1 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1460 to 2918
Columns: 103 entries, Id to outlier
dtypes: float64(16), int64(87)
memory usage: 1.2 MB


In [92]:
del data_train['outlier']
del data_toPredict['outlier']

In [93]:
del data_toPredict['SalePrice (box-cox)']

In [94]:
del data_toPredict['SalePrice']

In [95]:
x1 = data_train.drop(columns=["SalePrice", "SalePrice (box-cox)"]).values
y1 = data_train["SalePrice"].values
y1BC = data_train["SalePrice (box-cox)"].values

In [12]:
from tqdm import tqdm as progress_bar

## Model LineairRegression

In [13]:
def validation(x, y, *, model, k=10, log=False, desc=None):
    r2, rmse = [], []
    for train, test in progress_bar(KFold(n_splits=k).split(x), desc=desc, total=k):
        model.fit(x[train], y[train])
        y_predict = model.predict(x[test])
        y_true, y_predict = y[test], y_predict
        r2.append(r2_score(y_true, y_predict))
        rmse.append(mean_squared_error(y_true, y_predict))
    return {
        "r2": np.round(np.mean(r2), 3),
        "rmse": np.round(np.sqrt(np.mean(rmse))),
    }

In [14]:
results = {}

In [15]:
lr = LinearRegression()

In [16]:
results[('lr','o')] = validation(x1,y1, model=lr)

100%|██████████| 10/10 [00:00<00:00, 94.59it/s]


In [17]:
results[('lr','o')]

{'r2': 0.907, 'rmse': 22768.0}

In [132]:
def validationLog(x, y, *, model, k=10, log=False, desc=None):
    r2, rmse = [], []
    for train, test in progress_bar(KFold(n_splits=k).split(x), desc=desc, total=k):
        model.fit(x[train], y[train])
        y_predict = model.predict(x[test])
        y_true, y_predict = np.exp(y[test]), np.exp(y_predict)
        r2.append(r2_score(y_true, y_predict))
        rmse.append(mean_squared_error(y_true, y_predict))
    return {
        "r2": np.round(np.mean(r2), 3),
        "rmse": np.round(np.sqrt(np.mean(rmse))),
    }

In [133]:
results[("lr", "o")] = validationLog(x1, y1BC, model=lr)

100%|██████████| 10/10 [00:00<00:00, 77.13it/s]


In [134]:
results[('lr','o')]

{'r2': 0.925, 'rmse': 20533.0}

In [21]:
predictionKaggle = lr.predict(data_toPredict)

In [22]:
predictionKaggle = np.exp(predictionKaggle)

In [23]:
predictionKaggle

array([116067.61133056, 157985.40541571, 179378.6469482 , ...,
       179866.26424171, 120259.29664593, 228374.41396592])

In [24]:
data_toPredict['SalePrice'] = predictionKaggle.tolist()

<ipython-input-24-702b38bf4e9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_toPredict['SalePrice'] = predictionKaggle.tolist()


In [25]:
submission = data_toPredict[['Id','SalePrice']]

In [31]:
submission = submission.set_index('Id')

In [32]:
submission.head()

,SalePrice
Id,
1461,116067.611331
1462,157985.405416
1463,179378.646948
1464,196118.530364
1465,202641.125030


In [33]:
submission.to_csv("C:/Users/meykevandenbos/Documents/Meyke persoonlijk/Techionista/ML/House Prices/submission17022021-1.csv")

## Model Lasso

In [39]:
grid_search = GridSearchCV(
    estimator=Lasso(tol=0.1, selection="random", random_state=None),
    param_grid={"alpha": [2 ** x for x in range(-8, 4)] + list(range(12, 65, 4))},
    cv=KFold(n_splits=4),
    n_jobs=-1,
)

In [40]:
grid_search.fit(x1, y1)
alpha = grid_search.best_params_["alpha"]
alpha

56

In [43]:
results[("lasso", "o")] = validation(x1, y1, model=Lasso(alpha=alpha, tol=0.1))
results[("lasso", "o")]

100%|██████████| 10/10 [00:00<00:00, 91.48it/s]


{'r2': 0.881, 'rmse': 25869.0}

In [47]:
grid_search.fit(x1, y1BC)
alpha = grid_search.best_params_["alpha"]
alpha

0.00390625

In [48]:
results[("lasso", "o")] = validationLog(x1, y1BC, model=Lasso(alpha=alpha, tol=0.1))
results[("lasso", "o")]

100%|██████████| 10/10 [00:00<00:00, 107.19it/s]


{'r2': 0.908, 'rmse': 22689.0}

## Model Ridge

In [49]:
grid_search = GridSearchCV(
    estimator=Ridge(),
    param_grid={"alpha": [2 ** x for x in range(-8, 4)] + list(range(12, 65, 4))},
    cv=KFold(n_splits=4),
    n_jobs=-1,
)

In [50]:
grid_search.fit(x1, y1)
alpha = grid_search.best_params_["alpha"]
alpha

2

In [51]:
results[("ridge", "o")] = validation(x1, y1, model=Ridge(alpha=alpha))
results[("ridge", "o")]

100%|██████████| 10/10 [00:00<00:00, 134.66it/s]


{'r2': 0.909, 'rmse': 22609.0}

In [52]:
grid_search.fit(x1, y1BC)
alpha = grid_search.best_params_["alpha"]
alpha

4

In [102]:
results[("ridge", "o")] = validationLog(x1, y1BC, model=Ridge(alpha=alpha))
results[("ridge", "o")]

100%|██████████| 10/10 [00:00<00:00, 117.83it/s]


{'r2': 0.927, 'rmse': 20288.0}

In [107]:
def validationLog2(x, y, *, model, k=10, log=False, desc=None, data_toPredict):
    r2, rmse = [], []
    for train, test in progress_bar(KFold(n_splits=k).split(x), desc=desc, total=k):
        model.fit(x[train], y[train])
        y_predict = model.predict(x[test])
        y_true, y_predict = np.exp(y[test]), np.exp(y_predict)
        prediction = np.exp(model.predict(data_toPredict))
        r2.append(r2_score(y_true, y_predict))
        rmse.append(mean_squared_error(y_true, y_predict))
    return(prediction)

In [108]:
results[("ridge", "o")] = validationLog2(x1, y1BC, model=Ridge(alpha=alpha), data_toPredict=data_toPredict)
results[("ridge", "o")]

100%|██████████| 10/10 [00:00<00:00, 69.85it/s]


array([116686.15637458, 156614.00696209, 179790.82807044, ...,
       180416.67057055, 120300.21356694, 229786.52434068])

In [109]:
data_toPredict['SalePrice'] = results[("ridge", "o")].tolist()

<ipython-input-109-f280ee53fb77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_toPredict['SalePrice'] = results[("ridge", "o")].tolist()


In [110]:
submission = data_toPredict[['Id','SalePrice']]

In [111]:
submission = submission.set_index('Id')

In [112]:
submission.head()

,SalePrice
Id,
1461,116686.156375
1462,156614.006962
1463,179790.828070
1464,196265.578120
1465,199546.801788


In [113]:
submission.to_csv("C:/Users/meykevandenbos/Documents/Meyke persoonlijk/Techionista/ML/House Prices/submission17022021-3.csv")